In [60]:
import findspark
findspark.init()

from pyspark import *
from pyspark.sql.functions import desc, col, rand
from pyspark.sql import *
from graphframes import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from tqdm import tqdm
import copy
from math import comb

import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import sys
from sympy.ntheory.generate import nextprime
import time

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# https://graphframes.github.io/graphframes/docs/_site/quick-start.html
# https://stackoverflow.com/questions/65011599/how-to-start-graphframes-on-spark-on-pyspark-on-juypter-on-docker
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell'
os.environ['PYSPARK_PYTHON'] = sys.executable

In [4]:
spark = SparkSession.builder.appName('hw2') \
        .master("local[2]").getOrCreate()

21/11/18 16:30:41 WARN Utils: Your hostname, mark-machine resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface wlp8s0)
21/11/18 16:30:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mark/.ivy2/cache
The jars for the packages stored in: /home/mark/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-48951606-1d93-4910-aa95-07f32344e9f1;1.0
	confs: [default]
	found graphframes#graphframes;0.8.1-spark3.0-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 126ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.1-spark3.0-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------------

In [5]:
data_path = "data/T10I4D100K.dat"

In [6]:
def read_data(data_path):
    # read raw data
    data_raw_rdd = spark.sparkContext.textFile(data_path)
    # transform raw data to list of lists (list of ordered baskets)
    data_rdd = data_raw_rdd.map(lambda x: set([int(y) for y in x.strip().split(" ")]))
    # get item counts (k:v where k is hashed item id and v is count)
    items_counts_rdd = data_rdd.flatMap(lambda list: list).map(lambda w: (w,1)).reduceByKey(lambda a, b: a+b)
    # get items
    items_rdd = items_counts_rdd.map(lambda x: x[0])
    
    return data_rdd, items_rdd, items_counts_rdd

In [7]:
data_rdd, items_rdd, items_counts_rdd = read_data(data_path=data_path)

In [8]:
print(data_rdd.take(2))
n_baskets = data_rdd.count()
print(f"n_baskets = {n_baskets}")

[{448, 834, 164, 775, 328, 687, 240, 368, 274, 561, 52, 630, 730, 825, 538, 25}, {704, 834, 581, 39, 205, 814, 401, 120, 825, 124}]
n_baskets = 100000


In [9]:
items_rdd.take(20)
n_items = items_rdd.count()
print(f"n_items = {n_items}")

n_items = 870


In [10]:
print(items_counts_rdd.count())
items_counts_rdd.take(10)

870


[(448, 1370),
 (834, 1373),
 (164, 744),
 (328, 663),
 (240, 1399),
 (368, 7828),
 (274, 2628),
 (52, 1983),
 (630, 1523),
 (538, 3982)]

In [11]:
import itertools

def get_singletons(items_counts_rdd, s):
    singletons_rdd = items_counts_rdd.filter(lambda x: s <= x[1])
    singletons_rdd = singletons_rdd.map(lambda x: (set([x[0]]), x[1]))
    return singletons_rdd

def construct_itemsets(k, itemsets_frequent_rdd):
    assert 1 < k
    singletons_rdd = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == 1)
    k_minus_1_tons_rdd = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == k-1)
    k_minus_1_tons_rdd_collected = k_minus_1_tons_rdd.collect()
    # make tuple of itemsets of size k
    # return itemsets_candidates_rdd

def filter_itemsets(itemsets_candidates_rdd, k, s):
    pass
    # for each itemset of size k (itemset_k) in itemsets of size k, for each basket in baskets, 
    # if intersection of itemset_k and basket is equal to k, support counter of itemset_k +1
    
    # for itemset_k in itemset_ks, if support counter of itemset_k is >= s, keep, otherwise drop

In [12]:
itemsets_frequent_rdd = spark.sparkContext.parallelize([])

In [13]:
itemsets_frequent_rdd.collect()

[]

In [14]:
s = 1000
itemsets_frequent_rdd = get_singletons(items_counts_rdd=items_counts_rdd, s=s)
itemsets_frequent_rdd.count()

375

In [15]:
itemsets_frequent_rdd.take(10)

[({448}, 1370),
 ({834}, 1373),
 ({240}, 1399),
 ({368}, 7828),
 ({274}, 2628),
 ({52}, 1983),
 ({630}, 1523),
 ({538}, 3982),
 ({704}, 1794),
 ({814}, 1672)]

In [16]:
k = 2
singletons_rdd = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == 1)
k_minus_1_tons_rdd = itemsets_frequent_rdd.filter(lambda x: len(x[0]) == k-1)

In [17]:
singletons_rdd.take(10)

[({448}, 1370),
 ({834}, 1373),
 ({240}, 1399),
 ({368}, 7828),
 ({274}, 2628),
 ({52}, 1983),
 ({630}, 1523),
 ({538}, 3982),
 ({704}, 1794),
 ({814}, 1672)]

In [18]:
k_minus_1_tons_rdd.take(10)

[({448}, 1370),
 ({834}, 1373),
 ({240}, 1399),
 ({368}, 7828),
 ({274}, 2628),
 ({52}, 1983),
 ({630}, 1523),
 ({538}, 3982),
 ({704}, 1794),
 ({814}, 1672)]

In [19]:
l1 = singletons_rdd.map(lambda x: x[0])
l2 = k_minus_1_tons_rdd.map(lambda x: x[0])

In [20]:
l3 = l1.cartesian(l2)

In [21]:
l3.take(10)

[({448}, {448}),
 ({448}, {834}),
 ({448}, {240}),
 ({448}, {368}),
 ({448}, {274}),
 ({448}, {52}),
 ({448}, {630}),
 ({448}, {538}),
 ({448}, {704}),
 ({448}, {814})]

In [22]:
l4 = l3.map(lambda x: x[0].union(x[1])).filter(lambda x: len(x) == 2)

In [23]:
l4.take(200)

[{448, 834},
 {240, 448},
 {368, 448},
 {274, 448},
 {52, 448},
 {448, 630},
 {448, 538},
 {448, 704},
 {448, 814},
 {120, 448},
 {448, 674},
 {448, 854},
 {448, 950},
 {448, 964},
 {422, 448},
 {448, 738},
 {448, 708},
 {294, 448},
 {448, 966},
 {448, 978},
 {448, 766},
 {104, 448},
 {448, 620},
 {448, 798},
 {448, 682},
 {448, 970},
 {448, 782},
 {448, 658},
 {214, 448},
 {350, 448},
 {390, 448},
 {448, 530},
 {448, 914},
 {280, 448},
 {448, 932},
 {192, 448},
 {208, 448},
 {448, 720},
 {448, 618},
 {448, 496},
 {448, 706},
 {448, 878},
 {276, 448},
 {448, 960},
 {424, 448},
 {448, 490},
 {448, 910},
 {130, 448},
 {392, 448},
 {448, 862},
 {448, 900},
 {78, 448},
 {448, 778},
 {448, 572},
 {290, 448},
 {448, 614},
 {266, 448},
 {448, 458},
 {448, 944},
 {448, 888},
 {448, 480},
 {70, 448},
 {448, 874},
 {204, 448},
 {334, 448},
 {448, 504},
 {448, 890},
 {448, 810},
 {448, 844},
 {448, 846},
 {448, 722},
 {310, 448},
 {448, 918},
 {326, 448},
 {448, 774},
 {448, 526},
 {448, 788},
 {

In [24]:
l4.count()

140250

In [25]:
l5 = l4.map(lambda x: set(frozenset(x)))
l5c = l5.collect()
l5c2 = [set(item) for item in set(frozenset(item) for item in l5c)]
len(l5c2)

70125

In [26]:
l5c2

[{413, 494},
 {874, 978},
 {701, 946},
 {335, 804},
 {576, 583},
 {242, 684},
 {597, 641},
 {581, 766},
 {335, 538},
 {39, 884},
 {516, 854},
 {115, 735},
 {126, 952},
 {854, 895},
 {682, 740},
 {774, 984},
 {468, 984},
 {738, 749},
 {675, 790},
 {529, 600},
 {362, 789},
 {173, 527},
 {168, 948},
 {468, 912},
 {185, 266},
 {450, 572},
 {960, 991},
 {168, 309},
 {75, 606},
 {115, 797},
 {605, 910},
 {32, 357},
 {73, 413},
 {346, 600},
 {183, 274},
 {12, 419},
 {31, 548},
 {577, 631},
 {618, 632},
 {285, 632},
 {115, 981},
 {21, 944},
 {309, 381},
 {58, 541},
 {93, 561},
 {310, 812},
 {423, 978},
 {413, 628},
 {735, 984},
 {31, 638},
 {832, 910},
 {390, 510},
 {611, 765},
 {394, 684},
 {28, 571},
 {115, 975},
 {122, 214},
 {170, 948},
 {871, 966},
 {105, 623},
 {33, 769},
 {10, 41},
 {68, 227},
 {710, 823},
 {334, 472},
 {706, 722},
 {803, 947},
 {35, 234},
 {322, 885},
 {57, 258},
 {366, 522},
 {35, 853},
 {469, 550},
 {573, 890},
 {94, 140},
 {819, 944},
 {578, 966},
 {500, 964},
 {378

In [27]:
l5c2zip = [(idx, x) for idx, x in enumerate(l5c2)]
l5c2zip

[(0, {413, 494}),
 (1, {874, 978}),
 (2, {701, 946}),
 (3, {335, 804}),
 (4, {576, 583}),
 (5, {242, 684}),
 (6, {597, 641}),
 (7, {581, 766}),
 (8, {335, 538}),
 (9, {39, 884}),
 (10, {516, 854}),
 (11, {115, 735}),
 (12, {126, 952}),
 (13, {854, 895}),
 (14, {682, 740}),
 (15, {774, 984}),
 (16, {468, 984}),
 (17, {738, 749}),
 (18, {675, 790}),
 (19, {529, 600}),
 (20, {362, 789}),
 (21, {173, 527}),
 (22, {168, 948}),
 (23, {468, 912}),
 (24, {185, 266}),
 (25, {450, 572}),
 (26, {960, 991}),
 (27, {168, 309}),
 (28, {75, 606}),
 (29, {115, 797}),
 (30, {605, 910}),
 (31, {32, 357}),
 (32, {73, 413}),
 (33, {346, 600}),
 (34, {183, 274}),
 (35, {12, 419}),
 (36, {31, 548}),
 (37, {577, 631}),
 (38, {618, 632}),
 (39, {285, 632}),
 (40, {115, 981}),
 (41, {21, 944}),
 (42, {309, 381}),
 (43, {58, 541}),
 (44, {93, 561}),
 (45, {310, 812}),
 (46, {423, 978}),
 (47, {413, 628}),
 (48, {735, 984}),
 (49, {31, 638}),
 (50, {832, 910}),
 (51, {390, 510}),
 (52, {611, 765}),
 (53, {394, 6

In [28]:
import multiprocessing as mp

In [33]:
nprocs = mp.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 8


In [34]:
pool = mp.Pool(processes=nprocs)

In [35]:
data_rdd_c = data_rdd.collect()

In [36]:

#data_rdd.cache()
import time

start_time = time.time()

def f(candidate, data_rdd_c):
    return len(list(filter(lambda x: len(x) == 2, map(lambda x: x & candidate, data_rdd_c))))
    #return data_rdd.map(lambda x: x & candidate).filter(lambda x: len(x) == 2).count()

supports = pool.starmap(f, [(c,data_rdd_c) for c in l5c2])

end_time = time.time()

print(f"t={end_time - start_time}")

t=594.9545760154724


In [37]:
len(supports)

70125

In [38]:
res = spark.sparkContext.parallelize(l5c2zip).filter(lambda x: 1000 <= supports[x[0]]).map(lambda x: (x[1], supports[x[0]]))

In [40]:
res.take(20)

[({368, 829}, 1194),
 ({390, 722}, 1042),
 ({789, 829}, 1194),
 ({704, 825}, 1102),
 ({39, 704}, 1107),
 ({227, 390}, 1049),
 ({368, 682}, 1193),
 ({217, 346}, 1336),
 ({39, 825}, 1187)]

In [44]:
supports_1 = copy.deepcopy(supports)

In [92]:
resc = res.collect()

In [46]:
l10 = singletons_rdd.map(lambda x: x[0])
l11 = res.map(lambda x: x[0])

In [48]:
l12 = l10.cartesian(l11)
l12.take(10)

[({448}, {368, 829}),
 ({448}, {390, 722}),
 ({448}, {789, 829}),
 ({448}, {704, 825}),
 ({834}, {368, 829}),
 ({240}, {368, 829}),
 ({834}, {390, 722}),
 ({834}, {789, 829}),
 ({240}, {390, 722}),
 ({240}, {789, 829})]

In [55]:
l13 = l12.map(lambda x: x[0].union(x[1])).filter(lambda x: len(x) == 3)

In [114]:
l13.take(10)

[{368, 448, 829},
 {390, 448, 722},
 {448, 789, 829},
 {448, 704, 825},
 {368, 829, 834},
 {240, 368, 829},
 {390, 722, 834},
 {789, 829, 834},
 {240, 390, 722},
 {240, 789, 829}]

In [115]:
l13.count()

3357

In [116]:
l14 = l13.map(lambda x: set(frozenset(x)))
l14c = l14.collect()
l14c2 = [set(item) for item in set(frozenset(item) for item in l14c)]
len(l14c2)

3352

In [117]:
l14c2_rdd = spark.sparkContext.parallelize(l14c2)

In [118]:
n_comb = comb(3,1)
singletons_rddc = singletons_rdd.collect()
l14c2_rdd2 = l14c2_rdd.map(lambda x: (x, sum([len(x.intersection(s[0])) == 1 for s in singletons_rddc])))\
    .filter(lambda x: n_comb == x[1])

In [119]:
l14c2_rdd2.count()

3352

In [120]:
n_comb = comb(3,2)
l14c2_rdd3 = l14c2_rdd2.map(lambda x: (x[0], sum([len(x[0].intersection(s[0])) == 2 for s in resc])))\
    .filter(lambda x: n_comb == x[1])

In [122]:
l14c2_rdd3.count()

[({39, 704, 825}, 3)]

In [123]:
l14c2_rdd3.take(50)

[({39, 704, 825}, 3)]

In [124]:
l14c2_rdd3c = l14c2_rdd3.map(lambda x: x[0]).collect()
l14c2_rdd3c[:10]

[{39, 704, 825}]

In [159]:

#data_rdd.cache()
import time
pool = mp.Pool(processes=nprocs)
start_time = time.time()

def f(candidate, data_rdd_c):
    return len(list(filter(lambda x: len(x) == 3, map(lambda x: x & candidate, data_rdd_c))))
    #return data_rdd.map(lambda x: x & candidate).filter(lambda x: len(x) == 2).count()

supports3 = pool.starmap(f, [(c,data_rdd_c) for c in l14c2_rdd3c])

end_time = time.time()

print(f"t={end_time - start_time}")

t=0.6972157955169678


In [160]:
supports3

[1035]

In [161]:
l14c2_rdd3czip = [(idx, x) for idx, x in enumerate(l14c2_rdd3c)]
l14c2_rdd3czip

[(0, {39, 704, 825})]

In [162]:
res3 = spark.sparkContext.parallelize(l14c2_rdd3czip).filter(lambda x: 1000 <= supports3[x[0]]).map(lambda x: (x[1], supports3[x[0]]))

In [163]:
res3.take(50)

[({39, 704, 825}, 1035)]

In [164]:
len(data_rdd_c)

100000